# NYC Apartment Search

_[Project prompt](https://docs.google.com/document/d/1BYVyFBDcTywdUlanH0ysfOrNWPgl7UkqXA7NeewTzxA/edit#heading=h.bpxu7uvknnbk)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add code as you wish._

_**All code below should be consider "pseudo-code" - not functional by itself, and only an idea of a possible approach.**_

## Setup

In [1]:
# All import statements needed for the project, for example:

import json
import pathlib
import urllib.parse

# import geoalchemy2 as gdb
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import requests
import pip
import sqlalchemy as db

from sqlalchemy.orm import declarative_base

/Users/ljy/.conda/envs/untitled/lib/python3.10/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [2]:
# Any constants you might need; some have been added for you

# Where data files will be read from/written to - this should already exist
DATA_DIR = pathlib.Path("data")
ZIPCODE_DATA_FILE = DATA_DIR / "zipcodes" / "ZIP_CODE_040114.shp"
ZILLOW_DATA_FILE = DATA_DIR / "zillow_rent_data.csv"

NYC_DATA_APP_TOKEN = "FILL_ME_IN"
BASE_NYC_DATA_URL = "https://data.cityofnewyork.us/"
NYC_DATA_311 = "erm2-nwe9.geojson"
NYC_DATA_TREES = "5rq2-4hqu.geojson"

DB_NAME = "FILL_ME_IN"
DB_USER = "FILL_ME_IN"
DB_URL = f"postgres+psycopg2://{DB_USER}@localhost/{DB_NAME}"
DB_SCHEMA_FILE = "schema.sql"
# directory where DB queries for Part 3 will be saved
QUERY_DIR = pathlib.Path("queries")

In [3]:
# Make sure the QUERY_DIRECTORY exists
if not QUERY_DIR.exists():
    QUERY_DIR.mkdir()

In [4]:
app_token='Ynj6cS7u6dCNSS8Mx24yo8QLg'

## Part 1: Data Preprocessing

In [5]:
def download_nyc_geojson_data(url, force=False):
    parsed_url = urllib.parse.urlparse(url)
    url_path = parsed_url.path.strip("/")
    
    filename = DATA_DIR / url_path
    
    if force or not filename.exists():
        print(f"Downloading {url} to {filename}...")
        
        ...
        
        with open(filename, "w") as f:
            json.dump(..., f)
        print(f"Done downloading {url}.")

    else:
        print(f"Reading from {filename}...")

    return filename

In [5]:
def load_and_clean_zipcodes(zipcode_datafile):
    geodf_zipcode_data=gpd.read_file(zipcode_datafile)
    return geodf_zipcode_data
geodf_zipcode_data=load_and_clean_zipcodes('data/nyc_zipcodes/nyc_zipcodes.dbf')
geodf_zipcode_data


,ZIPCODE,BLDGZIP,PO_NAME,POPULATION,AREA,STATE,COUNTY,ST_FIPS,CTY_FIPS,URL,SHAPE_AREA,SHAPE_LEN,geometry
0,11436,0,Jamaica,18681.0,2.269930e+07,NY,Queens,36,081,http://www.usps.com/,0.0,0.0,"POLYGON ((1038098.252 188138.380, 1038141.936 ..."
1,11213,0,Brooklyn,62426.0,2.963100e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((1001613.713 186926.440, 1002314.243 ..."
2,11212,0,Brooklyn,83866.0,4.197210e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((1011174.276 183696.338, 1011373.584 ..."
3,11225,0,Brooklyn,56527.0,2.369863e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((995908.365 183617.613, 996522.848 18..."
4,11218,0,Brooklyn,72280.0,3.686880e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((991997.113 176307.496, 992042.798 17..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,10310,0,Staten Island,25003.0,5.346328e+07,NY,Richmond,36,085,http://www.usps.com/,0.0,0.0,"POLYGON ((950767.507 172848.969, 950787.510 17..."
259,11693,0,Far Rockaway,11052.0,3.497516e+06,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((1028453.995 167153.410, 1027813.010 ..."
260,11249,0,Brooklyn,28481.0,1.777221e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((995877.318 203206.075, 995968.511 20..."
261,10162,1,New York,0.0,2.103489e+04,NY,New York,36,061,http://www.usps.com/,0.0,0.0,"POLYGON ((997731.761 219560.922, 997641.948 21..."


In [7]:
geodf_zipcode_data=geodf_zipcode_data.drop(['BLDGZIP','POPULATION','AREA','ST_FIPS','CTY_FIPS','URL','SHAPE_AREA','SHAPE_LEN'],axis=1)
geodf_zipcode_data= geodf_zipcode_data.to_crs(epsg=4326)
geodf_zipcode_data

,ZIPCODE,PO_NAME,STATE,COUNTY,geometry
0,11436,Jamaica,NY,Queens,"POLYGON ((-73.80585 40.68291, -73.80569 40.682..."
1,11213,Brooklyn,NY,Kings,"POLYGON ((-73.93740 40.67973, -73.93487 40.679..."
2,11212,Brooklyn,NY,Kings,"POLYGON ((-73.90294 40.67084, -73.90223 40.668..."
3,11225,Brooklyn,NY,Kings,"POLYGON ((-73.95797 40.67066, -73.95576 40.670..."
4,11218,Brooklyn,NY,Kings,"POLYGON ((-73.97208 40.65060, -73.97192 40.650..."
...,...,...,...,...,...
258,10310,Staten Island,NY,Richmond,"POLYGON ((-74.12065 40.64104, -74.12057 40.641..."
259,11693,Far Rockaway,NY,Kings,"POLYGON ((-73.84076 40.62536, -73.84306 40.627..."
260,11249,Brooklyn,NY,Kings,"POLYGON ((-73.95805 40.72442, -73.95772 40.724..."
261,10162,New York,NY,New York,"POLYGON ((-73.95133 40.76931, -73.95165 40.769..."


In [ ]:
def download_and_clean_311_data():
    api_endpoint = 'https://data.cityofnewyork.us/resource/erm2-nwe9.json'
    selected_columns=['created_date','complaint_type','descriptor','incident_zip','latitude','longitude']
    params = {
    '$$app_token': app_token,
    '$where': 'created_date >= "2015-01-01T00:00:00.000"',
    '$select': ','.join(selected_columns),
    '$limit': 34856027


}
    response = requests.get(api_endpoint, params=params)
    if response.status_code == 200:
    # Load the data into a pandas DataFrame
        nyc_data = pd.read_json(response.text)

    # Now you have the data and can perform further processing
        nyc_data.to_csv('data/nyc_data_311.csv', index=False)
        return nyc_data
    else:
        print(f"Error: {response.status_code}, {response.text}")


geodf_311_data=download_and_clean_311_data()
geodf_311_data

In [9]:
geodf_311_data=pd.read_csv('data/nyc_data_311.csv',header=0)
geodf_311_data=geodf_311_data.dropna()
geodf_311_data=geodf_311_data[geodf_311_data['complaint_type'].str.contains('Noise')]
geodf_311_data

/var/folders/hf/mm0pmnq505ncq5b4sz9q0dz40000gn/T/ipykernel_21970/50730798.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  geodf_311_data=pd.read_csv('data/nyc_data_311.csv',header=0)


,created_date,complaint_type,descriptor,incident_zip,latitude,longitude
4,2023-11-29T01:19:45.000,Noise - Helicopter,Other,10028.0,40.780633,-73.960360
6,2023-11-29T01:18:29.000,Noise - Commercial,Loud Music/Party,10002.0,40.712416,-73.994366
7,2023-11-29T01:17:13.000,Noise - Helicopter,News Gathering,10025.0,40.798139,-73.968245
8,2023-11-29T01:16:49.000,Noise - Helicopter,Other,11231.0,40.680371,-73.990676
9,2023-11-29T01:16:13.000,Noise - Commercial,Loud Music/Party,11216.0,40.686815,-73.954611
...,...,...,...,...,...,...
24895793,2015-01-01T00:05:12.000,Noise - Residential,Loud Music/Party,11216,40.681473,-73.952790
24895794,2015-01-01T00:05:05.000,Noise - Street/Sidewalk,Loud Music/Party,10002,40.721235,-73.987770
24895795,2015-01-01T00:04:51.000,Noise - Residential,Loud Music/Party,11375,40.709232,-73.853204
24895797,2015-01-01T00:04:28.000,Noise - Vehicle,Car/Truck Horn,10468,40.867830,-73.907178


In [8]:
def download_and_clean_tree_data():
    api_endpoint1 = 'https://data.cityofnewyork.us/resource/5rq2-4hqu.geojson'
    selected_columns = ['tree_id','zipcode','latitude','longitude']
    params = {
        '$$app_token': app_token,
        '$limit': 683788,
        '$select': ','.join(selected_columns),
    }
    response = requests.get(api_endpoint1, params=params)
    if response.status_code == 200:
        # Load the data into a GeoDataFrame using geopandas
        geodf_tree_data  = gpd.read_file(response.text)

        # Save the GeoDataFrame to a CSV file
        geodf_tree_data.to_csv('data/tree_data.csv', index=False)

        return geodf_tree_data
    else:
        print(f"Error: {response.status_code}, {response.text}")

# 调用函数
geodf_tree_data = download_and_clean_tree_data()

In [10]:
geodf_tree_data=pd.read_csv('data/tree_data.csv')
geodf_tree_data=geodf_tree_data.drop(['geometry'],axis=1)
geodf_tree_data

,tree_id,zipcode,latitude,longitude
0,180683,11375,40.723092,-73.844215
1,200540,11357,40.794111,-73.818679
2,204026,11211,40.717581,-73.936608
3,204337,11211,40.713537,-73.934456
4,189565,11215,40.666778,-73.975979
...,...,...,...,...
683783,155433,11211,40.713211,-73.954944
683784,183795,11375,40.715194,-73.856650
683785,166161,10314,40.620762,-74.136517
683786,184028,10457,40.850828,-73.903115


In [11]:
def load_and_clean_zillow_data():
    df_zillow_data=pd.read_csv('data/zillow_rent_data.csv')
    df_zillow_data= df_zillow_data.drop(['RegionID','SizeRank','RegionType','StateName'],axis=1)
    df_zillow_data=df_zillow_data[df_zillow_data['City']=='New York']
    return df_zillow_data

df_zillow_data=load_and_clean_zillow_data()
df_zillow_data

,RegionName,State,City,Metro,CountyName,2015-01-31,2015-02-28,2015-03-31,2015-04-30,2015-05-31,...,2022-12-31,2023-01-31,2023-02-28,2023-03-31,2023-04-30,2023-05-31,2023-06-30,2023-07-31,2023-08-31,2023-09-30
4,11385,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,NaN,2087.527084,NaN,2149.924252,2166.263698,...,2935.808220,2895.699421,2873.209025,2881.906361,2913.546218,2963.964134,3005.735342,3034.413822,3064.476503,3079.585783
6,11208,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Kings County,NaN,NaN,NaN,NaN,NaN,...,2508.670432,2588.030194,2613.790654,2585.561351,2633.200754,2672.038493,2806.918757,2765.224364,2737.547470,2728.733333
12,11236,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Kings County,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2285.460026,2362.500000
13,10467,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Bronx County,NaN,NaN,NaN,NaN,NaN,...,2145.642295,2155.617718,2172.346611,2160.962748,2110.533203,2180.323655,2276.372290,2334.204728,2353.686402,2423.888889
14,11373,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,NaN,NaN,NaN,NaN,NaN,...,2199.459063,2255.604528,2262.101623,2271.514956,2250.182334,2231.959479,2257.413993,2247.592851,2302.557354,2292.994444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6550,10282,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",New York County,NaN,NaN,NaN,NaN,NaN,...,7058.613416,7143.347680,7408.940270,7459.080381,7512.509440,7452.333390,7572.806191,7611.834625,7444.236260,7347.458333
6561,11109,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,NaN,NaN,NaN,NaN,NaN,...,4247.334177,4159.759749,4157.528579,4201.199144,4294.386817,4343.065214,4357.893069,4445.207586,4490.367074,4529.358974
6644,10006,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",New York County,NaN,NaN,NaN,NaN,NaN,...,3936.629997,3747.416799,3693.121513,3812.835588,3920.619972,4025.134283,4043.838704,4035.676503,4042.831474,4060.096154
6695,10162,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",New York County,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4871.181752,5007.415824,NaN,4984.693932,5011.666667


In [12]:
# Show basic info about each dataframe
geodf_zipcode_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 263 entries, 0 to 262
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   ZIPCODE   263 non-null    object  
 1   PO_NAME   263 non-null    object  
 2   STATE     263 non-null    object  
 3   COUNTY    263 non-null    object  
 4   geometry  263 non-null    geometry
dtypes: geometry(1), object(4)
memory usage: 10.4+ KB


In [13]:
# Show first 5 entries about each dataframe
geodf_zipcode_data.head()

,ZIPCODE,PO_NAME,STATE,COUNTY,geometry
0,11436,Jamaica,NY,Queens,"POLYGON ((-73.80585 40.68291, -73.80569 40.682..."
1,11213,Brooklyn,NY,Kings,"POLYGON ((-73.93740 40.67973, -73.93487 40.679..."
2,11212,Brooklyn,NY,Kings,"POLYGON ((-73.90294 40.67084, -73.90223 40.668..."
3,11225,Brooklyn,NY,Kings,"POLYGON ((-73.95797 40.67066, -73.95576 40.670..."
4,11218,Brooklyn,NY,Kings,"POLYGON ((-73.97208 40.65060, -73.97192 40.650..."


In [14]:
geodf_311_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5079854 entries, 4 to 24895798
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   created_date    object 
 1   complaint_type  object 
 2   descriptor      object 
 3   incident_zip    object 
 4   latitude        float64
 5   longitude       float64
dtypes: float64(2), object(4)
memory usage: 271.3+ MB


In [15]:
geodf_311_data.head()

,created_date,complaint_type,descriptor,incident_zip,latitude,longitude
4,2023-11-29T01:19:45.000,Noise - Helicopter,Other,10028.0,40.780633,-73.960360
6,2023-11-29T01:18:29.000,Noise - Commercial,Loud Music/Party,10002.0,40.712416,-73.994366
7,2023-11-29T01:17:13.000,Noise - Helicopter,News Gathering,10025.0,40.798139,-73.968245
8,2023-11-29T01:16:49.000,Noise - Helicopter,Other,11231.0,40.680371,-73.990676
9,2023-11-29T01:16:13.000,Noise - Commercial,Loud Music/Party,11216.0,40.686815,-73.954611


In [16]:
geodf_tree_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 683788 entries, 0 to 683787
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   tree_id    683788 non-null  int64  
 1   zipcode    683788 non-null  int64  
 2   latitude   683788 non-null  float64
 3   longitude  683788 non-null  float64
dtypes: float64(2), int64(2)
memory usage: 20.9 MB


In [17]:
geodf_tree_data.head()

,tree_id,zipcode,latitude,longitude
0,180683,11375,40.723092,-73.844215
1,200540,11357,40.794111,-73.818679
2,204026,11211,40.717581,-73.936608
3,204337,11211,40.713537,-73.934456
4,189565,11215,40.666778,-73.975979


In [18]:
df_zillow_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 145 entries, 4 to 6721
Columns: 110 entries, RegionName to 2023-09-30
dtypes: float64(105), int64(1), object(4)
memory usage: 125.7+ KB


In [19]:
df_zillow_data.head()

,RegionName,State,City,Metro,CountyName,2015-01-31,2015-02-28,2015-03-31,2015-04-30,2015-05-31,...,2022-12-31,2023-01-31,2023-02-28,2023-03-31,2023-04-30,2023-05-31,2023-06-30,2023-07-31,2023-08-31,2023-09-30
4,11385,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,NaN,2087.527084,NaN,2149.924252,2166.263698,...,2935.808220,2895.699421,2873.209025,2881.906361,2913.546218,2963.964134,3005.735342,3034.413822,3064.476503,3079.585783
6,11208,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Kings County,NaN,NaN,NaN,NaN,NaN,...,2508.670432,2588.030194,2613.790654,2585.561351,2633.200754,2672.038493,2806.918757,2765.224364,2737.547470,2728.733333
12,11236,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Kings County,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2285.460026,2362.500000
13,10467,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Bronx County,NaN,NaN,NaN,NaN,NaN,...,2145.642295,2155.617718,2172.346611,2160.962748,2110.533203,2180.323655,2276.372290,2334.204728,2353.686402,2423.888889
14,11373,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,NaN,NaN,NaN,NaN,NaN,...,2199.459063,2255.604528,2262.101623,2271.514956,2250.182334,2231.959479,2257.413993,2247.592851,2302.557354,2292.994444


## Part 2: Storing Data

In [ ]:
!createdb YOUR_DATABASE_NAME

In [ ]:
!psql --dbname YOUR_DATABASE_NAME -c 'CREATE EXTENSION postgis;'

In [ ]:
def setup_new_postgis_database(username, db_name):
    raise NotImplementedError()

In [ ]:
setup_new_postgis_database(DB_USER, DB_NAME)

### Creating Tables


These are just a couple of options to creating your tables; you can use one or the other, a different method, or a combination.

#### Option 1: SQL

In [ ]:
# if using SQL (as opposed to SQLAlchemy), define the SQL statements to create your 4 tables
ZIPCODE_SCHEMA = """
CREATE TABLE zipcodes (
    id SERIAL PRIMARY KEY,
    zipcode VARCHAR(10),
    geometry GEOMETRY(POLYGON, 4326)
);
"""

NYC_311_SCHEMA = """
CREATE TABLE complaints (
    id SERIAL PRIMARY KEY,
    complaint_type VARCHAR(255),
    geometry GEOMETRY(POINT, 4326)
);
"""

NYC_TREE_SCHEMA = """
CREATE TABLE trees (
    id SERIAL PRIMARY KEY,
    tree_id INTEGER,
    geometry GEOMETRY(POINT, 4326)
);
"""

ZILLOW_SCHEMA = """
CREATE TABLE rents (
    id SERIAL PRIMARY KEY,
    zipcode VARCHAR(10),
    month DATE,
    rent_avg NUMERIC
);
"""

In [ ]:
# create that required schema.sql file
with open(DB_SCHEMA_FILE, "w") as f:
    f.write(ZIPCODE_SCHEMA)
    f.write(NYC_311_SCHEMA)
    f.write(NYC_TREE_SCHEMA)
    f.write(ZILLOW_SCHEMA)

In [ ]:
# If using SQL (as opposed to SQLAlchemy), execute the schema files to create tables
with engine.connect() as connection:
    with connection.begin():
        connection.execute(ZIPCODE_SCHEMA)
        connection.execute(NYC_311_SCHEMA)
        connection.execute(NYC_TREE_SCHEMA)
        connection.execute(ZILLOW_SCHEMA)
    #pass

#### Option 2: SQLAlchemy

In [ ]:
Base = declarative_base()

class Tree(Base):
    __tablename__ = "trees"

    ...


In [ ]:
Base.metadata.create_all(engine)

### Add Data to Database

These are just a couple of options to write data to your tables; you can use one or the other, a different method, or a combination.

#### Option 1: SQL

In [ ]:
def write_dataframes_to_table(tablename_to_dataframe):
    # write INSERT statements or use pandas/geopandas to write SQL
    for table, df in tablename_to_dataframe.items():
        df.to_sql(table, engine, index=False, if_exists='replace', dtype={'geometry': gdb.Geometry('POINT', srid=4326)})
    raise NotImplemented()

In [ ]:
tablename_to_dataframe = {
    "zipcodes": geodf_zipcode_data,
    "complaints": geodf_311_data,
    "trees": geodf_tree_data,
    "rents": df_zillow_data,
}

In [ ]:
write_dataframes_to_table(tablename_to_dataframe)

In [ ]:
# Create indices on geometry columns
with engine.connect() as connection:
    connection.execute("CREATE INDEX idx_zipcodes_geom ON zipcodes USING GIST(geometry);")
    connection.execute("CREATE INDEX idx_complaints_geom ON complaints USING GIST(geometry);")
    connection.execute("CREATE INDEX idx_trees_geom ON trees USING GIST(geometry);")
    connection.execute("CREATE INDEX idx_rents_geom ON rents USING GIST(geometry);")

#### Option 2: SQLAlchemy

In [ ]:
Session = db.orm.sessionmaker(bind=engine)
session = Session()

In [ ]:
for row in geodf_tree_data.iterrows():
    tree = Tree(...)
    session.add(tree)

In [ ]:
session.commit()

## Part 3: Understanding the Data

### Query 1

In [ ]:
# Helper function to write the queries to file
def write_query_to_file(query, outfile):
    with open(outfile, "w") as f:
        f.write(query)

In [ ]:
QUERY_1_FILENAME = QUERY_DIR / "FILL_ME_IN"

QUERY_1 = """
SELECT zipcode, COUNT(*) as num_complaints
FROM complaints
WHERE date BETWEEN '2022-10-01' AND '2023-09-30'
GROUP BY zipcode
ORDER BY num_complaints DESC;
"""

QUERY_1_FILENAME = QUERY_DIR / "top_10_zipcodes_by_complaints.sql"

In [ ]:
with engine.connect() as conn:
    result = conn.execute(db.text(QUERY_1))
    for row in result:
        print(row)

In [ ]:
write_query_to_file(QUERY_1, QUERY_1_FILENAME)

In [ ]:
QUERY_2 = """
SELECT zipcode, COUNT(*) as num_trees
FROM trees
GROUP BY zipcode
ORDER BY num_trees DESC
LIMIT 10;
"""

QUERY_2_FILENAME = QUERY_DIR / "top_10_zipcodes_by_trees.sql"

In [ ]:
with engine.connect() as conn:
    result = conn.execute(db.text(QUERY_2))
    for row in result:
        print(row)
write_query_to_file(QUERY_2, QUERY_2_FILENAME)

In [ ]:
QUERY_3 = """
SELECT t.zipcode, AVG(r.average_rent) as avg_rent
FROM trees t
JOIN rents r ON t.zipcode = r.zipcode AND r.month = '2023-08-01'
GROUP BY t.zipcode
ORDER BY COUNT(*) DESC
LIMIT 10;
"""

QUERY_3_FILENAME = QUERY_DIR / "avg_rent_in_top_10_zipcodes_with_trees.sql"

In [ ]:
with engine.connect() as conn:
    result = conn.execute(db.text(QUERY_3))
    for row in result:
        print(row)
write_query_to_file(QUERY_3, QUERY_3_FILENAME)

In [ ]:
QUERY_4 = """
WITH rents_tree_complaints AS (
    SELECT t.zipcode, AVG(r.average_rent) as avg_rent, COUNT(*) as num_trees, COUNT(c.*) as num_complaints
    FROM trees t
    JOIN rents r ON t.zipcode = r.zipcode AND r.month = '2023-01-01'
    LEFT JOIN complaints c ON t.zipcode = c.zipcode AND c.date BETWEEN '2023-01-01' AND '2023-01-31'
    GROUP BY t.zipcode
)
SELECT *
FROM (
    SELECT *, ROW_NUMBER() OVER (ORDER BY avg_rent ASC) as row_low
    FROM rents_tree_complaints
    WHERE avg_rent IS NOT NULL
) AS low
UNION ALL
SELECT *
FROM (
    SELECT *, ROW_NUMBER() OVER (ORDER BY avg_rent DESC) as row_high
    FROM rents_tree_complaints
    WHERE avg_rent IS NOT NULL
) AS high
WHERE row_low <= 5 OR row_high <= 5
ORDER BY avg_rent DESC, num_trees DESC, num_complaints DESC;
"""

QUERY_4_FILENAME = QUERY_DIR / "rent_tree_complaint_correlation.sql"

In [ ]:
with engine.connect() as conn:
    result = conn.execute(db.text(QUERY_4))
    for row in result:
        print(row)
write_query_to_file(QUERY_4, QUERY_4_FILENAME)

In [ ]:
QUERY_5 = """
SELECT z.zipcode, COUNT(*) as num_trees
FROM trees t
JOIN zipcodes z ON ST_Within(t.geometry, z.geometry)
GROUP BY z.zipcode
ORDER BY num_trees DESC
LIMIT 10;
"""

QUERY_5_FILENAME = QUERY_DIR / "top_10_zipcodes_by_trees_v2.sql"

In [ ]:
with engine.connect() as conn:
    result = conn.execute(db.text(QUERY_5))
    for row in result:
        print(row)
write_query_to_file(QUERY_5, QUERY_5_FILENAME)

In [ ]:
QUERY_6 = """
SELECT id, tree_species, health, status, ST_AsText(geometry) as coordinate_location
FROM trees
WHERE ST_DistanceSphere(ST_MakePoint(-73.96253174434912, 40.80737875669467), geometry) <= 804.672;  -- 0.5 mile in meters
"""

QUERY_6_FILENAME = QUERY_DIR / "trees_within_half_mile_radius.sql"

In [ ]:
with engine.connect() as conn:
    result = conn.execute(db.text(QUERY_6))
    for row in result:
        print(row)
write_query_to_file(QUERY_6, QUERY_6_FILENAME)

## Part 4: Visualizing the Data

### Visualization 1

In [ ]:
# use a more descriptive name for your function
def plot_visual_1(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_1():
    # Query your database for the data needed.
    # You can put the data queried into a pandas/geopandas dataframe, if you wish
    raise NotImplementedError()

In [ ]:
some_dataframe = get_data_for_visual_1()
plot_visual_1(some_dataframe)